# **Bitcoin price prediction - Generalized Linear Regression (Single Split)**
### Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author: Corsi Danilo (1742375) - corsi.1742375@studenti.uniroma1.it


---


Description: perform model's train / validation with hyperparameter tuning and cross validation based on different methods of splitting the dataset.

# Global constants, dependencies, libraries and tools

In [1]:
# Main constants
LOCAL_RUNNING = True
SLOW_OPERATIONS = False # Decide whether or not to use operations that might slow down notebook execution
ROOT_DIR = "D:/Documents/Repository/BDC/project" if LOCAL_RUNNING else "/content/drive"

In [2]:
if not LOCAL_RUNNING:
    # Point Colaboratory to Google Drive
    from google.colab import drive

    # Define GDrive paths
    drive.mount(ROOT_DIR, force_remount=True)

    # Install Spark and related dependencies
    !pip install pyspark
    !pip install -U -q PyDrive -qq
    !apt install openjdk-8-jdk-headless -qq

## Import my utilities

In [77]:
# Set main dir
MAIN_DIR = ROOT_DIR + "" if LOCAL_RUNNING else ROOT_DIR + "/MyDrive/BDC/project"

# Utilities dir
UTILITIES_DIR = MAIN_DIR + "/utilities"

# Import my utilities
import sys
sys.path.append(UTILITIES_DIR)

from imports import *
import train_validation_utilities
from config import *

importlib.reload(train_validation_utilities)

<module 'train_validation_utilities' from 'D:\\Documents/Repository/BDC/project/utilities\\train_validation_utilities.py'>

## Core variables

In [4]:
# BS = Block Split
# WFS = Walk Forward Split
# SS = Single Split
SPLITTING_METHOD = SS 

# LR = LinearRegression 
# GLR = GeneralizedLinearRegression 
# RF = RandomForestRegressor 
# GBTR = GradientBoostingTreeRegressor
MODEL_NAME = GLR

In [5]:
###################
# --- DATASET --- #
###################

# Datasets dirs
DATASET_OUTPUT_DIR = MAIN_DIR + "/datasets/output"

# Datasets paths
DATASET_TRAIN_VALID  = DATASET_OUTPUT_DIR + "/" + DATASET_TRAIN_VALID_NAME + ".parquet"

####################
# --- FEATURES --- #
####################

# Features dir
FEATURES_DIR = MAIN_DIR + "/features"

# Features paths
FEATURES_CORRELATION = FEATURES_DIR + "/" + FEATURES_CORRELATION_LABEL + ".json"
BASE_FEATURES = FEATURES_DIR + "/" + BASE_FEATURES_LABEL + ".json"
BASE_AND_MOST_CORR_FEATURES = FEATURES_DIR + "/" + BASE_AND_MOST_CORR_FEATURES_LABEL + ".json"
BASE_AND_LEAST_CORR_FEATURES = FEATURES_DIR + "/" + BASE_AND_LEAST_CORR_FEATURES_LABEL + ".json"

##################
# --- MODELS --- #
##################

# Model dir
MODELS_DIR = MAIN_DIR + "/models"

# Model path
MODEL = MODELS_DIR + "/" + MODEL_NAME

###################
# --- RESULTS --- #
###################

# Results dir
RESULTS_DIR = MAIN_DIR + "/results/" + SPLITTING_METHOD

# Results path
ALL_MODEL_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + "_all.csv"
REL_MODEL_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + "_rel.csv"

MODEL_ACCURACY_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + "_accuracy.csv"

In [6]:
# Importing useful libraries
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pio.renderers.default = 'vscode+colab' # To correctly render plotly plots

# Create the pyspark session

In [7]:
# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '12G').\
                set('spark.driver.memory', '12G').\
                set('spark.driver.maxResultSize', '109G').\
                set("spark.kryoserializer.buffer.max", "1G").\
                setAppName("BitcoinPricePrediction").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

# Loading dataset

In [8]:
# Load train / validation set into pyspark dataset objects
df = spark.read.load(DATASET_TRAIN_VALID,
                         format="parquet",
                         sep=",",
                         inferSchema="true",
                         header="true"
                    )

In [9]:
train_validation_utilities.dataset_info(df)

+-------------------+---+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+-----------------+--------------------+--------------------+------------------+------------------+--------------------+------------------------+-------------------+------------------+--------------------+--------------------+------------------+--------------+--------------------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|          timestamp| id|     market-price|    opening-price|    highest-price|     lowest-price|    closing-price|  trade-volume-btc|   total-bitcoins|          market-cap|    trade-volume-usd|       blocks-size|    avg-block-size|n-transactions-total|n-transactions-per-block|          hash-rate|        difficulty|      miners-revenue|transaction-fees-usd|n-unique-addresses|n-transactions|estimated-transaction-volume-usd|       sma-5-days|   

Instead of considering the entire dataset, I will only consider the last two years from today's date in order to train / validate the model.

In [10]:
# Retrieve the last timestamp value
last_value = df.agg(last("timestamp")).collect()[0][0]

# Consider only the last two years as of today
split_date = last_value - relativedelta(years=2)

# Split the dataset based on the desired date and drop the "id" column
df = df[df['timestamp'] > split_date].drop(col("id"))

# Recompute id column
df = df.withColumn("id", F.row_number().over(Window.orderBy(F.monotonically_increasing_id()))-1)
# Rearranges columns
new_columns = ["timestamp", "id"] + [col for col in df.columns if col not in ["timestamp", "id", "next-market-price"]] + ["next-market-price"]
df = df.select(*new_columns)

In [11]:
train_validation_utilities.dataset_info(df)

+-------------------+---+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+--------------------+--------------------+------------------+------------------+--------------------+------------------------+--------------------+--------------------+--------------------+--------------------+------------------+------------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|          timestamp| id|      market-price|     opening-price|     highest-price|      lowest-price|     closing-price|  trade-volume-btc|   total-bitcoins|          market-cap|    trade-volume-usd|       blocks-size|    avg-block-size|n-transactions-total|n-transactions-per-block|           hash-rate|          difficulty|      miners-revenue|transaction-fees-usd|n-unique-addresses|    n-transactions|estimated-transaction-v

# Loading features

In [12]:
# Loading base features
with open(BASE_FEATURES, "r") as f:
    BASE_FEATURES = json.load(f)
print(BASE_FEATURES)

['opening-price', 'highest-price', 'lowest-price', 'closing-price', 'trade-volume-btc', 'market-price', 'market-cap', 'total-bitcoins', 'trade-volume-usd']


In [13]:
# Loading currency and blockchain most correlated features
with open(BASE_AND_MOST_CORR_FEATURES, "r") as f:
    BASE_AND_MOST_CORR_FEATURES = json.load(f)
print(BASE_AND_MOST_CORR_FEATURES)

['opening-price', 'highest-price', 'lowest-price', 'closing-price', 'trade-volume-btc', 'market-price', 'market-cap', 'total-bitcoins', 'trade-volume-usd', 'miners-revenue', 'sma-5-days', 'sma-7-days', 'sma-10-days', 'estimated-transaction-volume-usd', 'sma-20-days']


In [14]:
# Loading currency and blockchain least correlated features
with open(BASE_AND_LEAST_CORR_FEATURES, "r") as f:
    BASE_AND_LEAST_CORR_FEATURES = json.load(f)
print(BASE_AND_LEAST_CORR_FEATURES)

['opening-price', 'highest-price', 'lowest-price', 'closing-price', 'trade-volume-btc', 'market-price', 'market-cap', 'total-bitcoins', 'trade-volume-usd', 'sma-100-days', 'transaction-fees-usd', 'n-unique-addresses', 'sma-50-days', 'n-transactions-total', 'blocks-size', 'hash-rate', 'difficulty', 'avg-block-size', 'n-transactions-per-block', 'n-transactions']


# Model train / validation
In order to train and validate the model I'll try several approaches:
- `Default without normalization:` make predictions using the base model
- `Default with normalization:` like the previous one but features are normalized

Then the features that gave on average the most satisfactory results (for each model) are chosen and proceeded with:
- `Hyperparameter tuning:` finding the best parameters to use. 
- `Cross Validation:` validate the performance of the model with the chosen parameters (also here using Block split / Walk forward split)

If the final results are satisfactory, the model will be trained on the whole train / validation set and saved in order to make predictions on the test set.

For each approach the train / validation set will be split according to the chosen splitting method (in order to figure out which one works best for our problem). In this case the `Single time series split` method will be used: involves dividing the time series considering as validation set a narrow period of time and as train set everything that happened before this period, in such a way as to best benefit from the trend in the short term.

<img src="https://github.com/CorsiDanilo/big-data-computing-project/blob/main/notebooks/images/single-split.png?raw=1">

In [15]:
# Get splitting parameters based on the choosen splitting method
splitting_info = train_validation_utilities.get_splitting_params(SPLITTING_METHOD)
splitting_info

{'split_type': 'single_split', 'split_label': 'months', 'split_value': 1}

## Default
The train / validation set will be splitted based on the splitting method chosen so that the model performance can be seen without any tuning by using different features (normalized and non)

In [16]:
# Get default parameters
params = train_validation_utilities.get_defaults_model_params(MODEL_NAME)
params

{'maxIter': [25], 'regParam': [0]}

### Without normalization

In [17]:
# Define model and features type
MODEL_TYPE = "default"
FEATURES_NORMALIZATION = False

In [18]:
# Choose base features
CHOSEN_FEATURES = BASE_FEATURES
CHOSEN_FEATURES_LABEL = BASE_FEATURES_LABEL

In [19]:
# Make predictions by using base features
default_train_results_base_features, default_valid_results_base_features, default_train_pred_base_features, default_valid_pred_base_features = train_validation_utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

In [20]:
default_train_results_base_features

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default,train,single_split,base_features,"(67104, 2976)","[25, 0]",9.296477,86.424484,5.969337,0.000177,0.999999,0.999999,1.813228


In [21]:
default_valid_results_base_features

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default,valid,single_split,base_features,"(67104, 2976)","[25, 0]",2.683494,7.201143,1.993841,0.000067,0.999958,0.999958,1.813228


In [22]:
# Choose base and most additional correlated features
CHOSEN_FEATURES = BASE_AND_MOST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_MOST_CORR_FEATURES_LABEL

In [23]:
# Make predictions by using base and most additional correlated features
default_train_results_base_and_most_corr_features, default_valid_results_base_and_most_corr_features, default_train_pred_base_and_most_corr_features, default_valid_pred_base_and_most_corr_features = train_validation_utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

In [24]:
default_train_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default,train,single_split,base_and_most_corr_features,"(67104, 2976)","[25, 0]",9.253502,85.627308,5.968947,0.000177,1.0,1.0,1.047293


In [25]:
default_valid_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default,valid,single_split,base_and_most_corr_features,"(67104, 2976)","[25, 0]",2.746864,7.545262,2.049422,0.000069,0.999956,0.999956,1.047293


In [26]:
# Choose base and least additional correlated features
CHOSEN_FEATURES = BASE_AND_LEAST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_LEAST_CORR_FEATURES_LABEL

In [27]:
# Make predictions by using base and least additional correlated features
default_train_results_base_and_least_corr_features, default_valid_results_base_and_least_corr_features, default_train_pred_base_and_least_corr_features, default_valid_pred_base_and_least_corr_features = train_validation_utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

In [28]:
default_train_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default,train,single_split,base_and_least_corr_features,"(67104, 2976)","[25, 0]",9.250705,85.575552,5.967845,0.000178,1.0,1.0,1.057417


In [29]:
default_train_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default,train,single_split,base_and_least_corr_features,"(67104, 2976)","[25, 0]",9.250705,85.575552,5.967845,0.000178,1.0,1.0,1.057417


### With normalization

In [30]:
# Define model and features type
MODEL_TYPE = "default_norm"
FEATURES_NORMALIZATION = True

In [31]:
# Choose base features
CHOSEN_FEATURES = BASE_FEATURES
CHOSEN_FEATURES_LABEL = BASE_FEATURES_LABEL

In [32]:
# Make predictions by using base features
default_norm_train_results_base_features, default_norm_valid_results_base_features, default_norm_train_pred_base_features, default_norm_valid_pred_base_features = train_validation_utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

In [33]:
default_norm_train_results_base_features

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default_norm,train,single_split,base_features_norm,"(67104, 2976)","[25, 0]",4076.944537,16621476.758112,3121.716988,0.107125,0.903166,0.90316,0.752858


In [34]:
default_norm_valid_results_base_features

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default_norm,valid,single_split,base_features_norm,"(67104, 2976)","[25, 0]",3225.273767,10402390.872995,3159.373044,0.106917,-59.78376,-59.865596,0.752858


In [35]:
# Choose base and most additional correlated features
CHOSEN_FEATURES = BASE_AND_MOST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_MOST_CORR_FEATURES_LABEL

In [36]:
# Make predictions by using base and most additional correlated features
default_norm_train_results_base_and_most_corr_features, default_norm_valid_results_base_and_most_corr_features, default_norm_train_pred_base_and_most_corr_features, default_norm_valid_pred_base_and_most_corr_features = train_validation_utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

In [37]:
default_norm_train_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default_norm,train,single_split,base_and_most_corr_features_norm,"(67104, 2976)","[25, 0]",3146.581954,9900977.992043,2551.31553,0.094561,0.942318,0.942315,0.551809


In [38]:
default_norm_valid_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default_norm,valid,single_split,base_and_most_corr_features_norm,"(67104, 2976)","[25, 0]",829.037237,687302.740443,672.362273,0.022838,-3.016081,-3.021488,0.551809


In [39]:
# Choose base and least additional correlated features
CHOSEN_FEATURES = BASE_AND_LEAST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_LEAST_CORR_FEATURES_LABEL

In [40]:
# Make predictions by using base and least additional correlated features
default_norm_train_results_base_and_least_corr_features, default_norm_valid_results_base_and_least_corr_features, default_norm_train_pred_base_and_least_corr_features, default_norm_valid_pred_base_and_least_corr_features = train_validation_utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

In [41]:
default_norm_train_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default_norm,train,single_split,base_and_least_corr_features_norm,"(67104, 2976)","[25, 0]",412.026663,169765.97092,310.156333,0.010545,0.999011,0.999011,0.609058


In [42]:
default_norm_valid_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default_norm,valid,single_split,base_and_least_corr_features_norm,"(67104, 2976)","[25, 0]",1586.55923,2517170.190764,1559.544555,0.052708,-13.708452,-13.728255,0.609058


In [43]:
# Define model information and evaluators to show
model_info = ['Model', 'Type', 'Dataset', 'Splitting', 'Features', 'Parameters']
evaluator_lst = ['RMSE', 'MSE', 'MAE', 'MAPE', 'R2', 'Adjusted_R2', 'Time']

In [44]:
# Define the results to show
default_comparison_lst = [default_valid_results_base_features, default_valid_results_base_and_most_corr_features, default_valid_results_base_and_least_corr_features, default_norm_valid_results_base_features, default_norm_valid_results_base_and_most_corr_features, default_norm_valid_results_base_and_least_corr_features]

# Show the comparison table
default_comparison_lst_df = pd.concat([train_validation_utilities.model_comparison(results, model_info, evaluator_lst) for results in default_comparison_lst])
default_comparison_lst_df

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default,valid,single_split,base_features,"[25, 0]",2.683494,7.201143e+00,1.993841,0.000067,0.999958,0.999958,1.813228
0,GeneralizedLinearRegression,default,valid,single_split,base_and_most_corr_features,"[25, 0]",2.746864,7.545262e+00,2.049422,0.000069,0.999956,0.999956,1.047293
0,GeneralizedLinearRegression,default,valid,single_split,base_and_least_corr_features,"[25, 0]",5.350591,2.862883e+01,4.869163,0.000165,0.999833,0.999832,1.057417
0,GeneralizedLinearRegression,default_norm,valid,single_split,base_features_norm,"[25, 0]",3225.273767,1.040239e+07,3159.373044,0.106917,-59.783760,-59.865596,0.752858
0,GeneralizedLinearRegression,default_norm,valid,single_split,base_and_most_corr_features_norm,"[25, 0]",829.037237,6.873027e+05,672.362273,0.022838,-3.016081,-3.021488,0.551809
0,GeneralizedLinearRegression,default_norm,valid,single_split,base_and_least_corr_features_norm,"[25, 0]",1586.559230,2.517170e+06,1559.544555,0.052708,-13.708452,-13.728255,0.609058


In [47]:
# Save the best default model results and predicitons
best_default_results = default_norm_valid_results_base_and_most_corr_features
best_default_predictions = default_norm_valid_pred_base_and_most_corr_features
best_default_results

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default_norm,valid,single_split,base_and_most_corr_features_norm,"(67104, 2976)","[25, 0]",829.037237,687302.740443,672.362273,0.022838,-3.016081,-3.021488,0.551809


In [87]:
# Define the choosen features
FEATURES_NORMALIZATION = True
CHOSEN_FEATURES = BASE_AND_MOST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_MOST_CORR_FEATURES_LABEL

## Tuned
Once the features and execution method are selected, the model will undergo hyperparameter tuning and cross validation to find the best configuration.

In [49]:
# Using one of the multiple splitting methods to tune the model
SPLITTING_METHOD = "walk_forward_splits"

# Get splitting parameters based on the choosen splitting method
splitting_info = train_validation_utilities.get_splitting_params(SPLITTING_METHOD)
splitting_info

{'split_type': 'walk_forward_splits',
 'min_obser': 20000,
 'sliding_window': 5000}

In [50]:
# Get model grid parameters
params = train_validation_utilities.get_model_grid_params(MODEL_NAME)
params

{'maxIter': [5, 10, 50, 80],
 'regParam': [0, 0.1, 0.2],
 'family': ['gaussian', 'gamma'],
 'link': ['log', 'identity', 'inverse']}

### Hyperparameter tuning

In [51]:
# Select the type of feature to be used
MODEL_TYPE = "hyp_tuning"

In [52]:
# Perform hyperparameter tuning
hyp_res = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)
hyp_res

Split: [1/10]: 100%|██████████| 72/72 [00:54<00:00,  1.33it/s]


Best parameters chosen for split [1/10]: [5, 0.2, 'gaussian', 'log']


Split: [2/10]: 100%|██████████| 72/72 [00:55<00:00,  1.30it/s]


Best parameters chosen for split [2/10]: [5, 0, 'gaussian', 'log']


Split: [3/10]: 100%|██████████| 72/72 [00:53<00:00,  1.36it/s]


...

Split: [9/10]: 100%|██████████| 72/72 [00:54<00:00,  1.32it/s]


Best parameters chosen for split [9/10]: [5, 0.2, 'gaussian', 'log']


Split: [10/10]: 100%|██████████| 72/72 [00:52<00:00,  1.37it/s]

Best parameters chosen for split [10/10]: [5, 0.2, 'gaussian', 'log']


,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,hyp_tuning,valid,walk_forward_splits,base_and_most_corr_features_norm,1,"(20000, 5000)","[5, 0.2, gaussian, log]",1032.505654,1.066068e+06,823.533878,0.019276,0.847744,0.847622,0.083831
1,GeneralizedLinearRegression,hyp_tuning,valid,walk_forward_splits,base_and_most_corr_features_norm,2,"(20000, 5000)","[5, 0, gaussian, log]",10952.673316,1.199611e+08,8031.665793,0.321014,-4.361458,-4.365751,0.305007
2,GeneralizedLinearRegression,hyp_tuning,valid,walk_forward_splits,base_and_most_corr_features_norm,3,"(20000, 5000)","[5, 0.2, gaussian, log]",13917.335912,1.936922e+08,10669.519722,0.470179,-93.140048,-93.215435,0.057583
3,GeneralizedLinearRegression,hyp_tuning,valid,walk_forward_splits,base_and_most_corr_features_norm,4,"(20000, 5000)","[5, 0.2, gaussian, log]",14241.104168,2.028090e+08,12366.473172,0.624681,-81.228460,-81.294309,0.058224
4,GeneralizedLinearRegression,hyp_tuning,valid,walk_forward_splits,base_and_most_corr_features_norm,5,"(20000, 5000)","[5, 0.2, gaussian, log]",5853.310216,3.426124e+07,5513.065360,0.293564,-12.360351,-12.371050,0.054587
5,GeneralizedLinearRegression,hyp_tuning,valid,walk_forward_splits,base_and_most_corr_features_norm,6,"(20000, 5000)","[5, 0.2, gaussian, log]",3851.819490,1.483651e+07,3546.354793,0.205909,-19.361102,-19.377407,0.054988
6,GeneralizedLinearRegression,hyp_tuning,valid,walk_forward_splits,base_and_most_corr_features_norm,7,"(20000, 5000)","[5, 0.2, gaussian, log]",305.463130,9.330772e+04,229.193086,0.009992,0.895024,0.894940,0.053041
7,GeneralizedLinearRegression,hyp_tuning,valid,walk_forward_splits,base_and_most_corr_features_norm,8,"(20000, 5000)","[5, 0, gaussian, log]",1995.510171,3.982061e+06,1848.847906,0.065382,0.332986,0.332452,0.361435
8,GeneralizedLinearRegression,hyp_tuning,valid,walk_forward_splits,base_and_most_corr_features_norm,9,"(20000, 5000)","[5, 0.2, gaussian, log]",2271.432082,5.159404e+06,1898.854260,0.071028,-4.672269,-4.676812,0.055998
9,GeneralizedLinearRegression,hyp_tuning,valid,walk_forward_splits,base_and_most_corr_features_norm,10,"(20000, 5000)","[5, 0.2, gaussian, log]",2073.920899,4.301148e+06,2043.932423,0.068153,-13.009683,-13.020901,0.052885


---
Since during this stage will be used the Block split / Walk forward split method of the dataset I compute a score for each parameter chosen by each split, assigning weights based on:
   * Their `frequency` for each split (if the same parameters are chosen from several splits, these will have greater weight) 
   * The `split` they belong to (the closer the split is to today's date the more weight they will have)
   * Their `RMSE value` for each split (the lower this is, the more weight they will have)
   
   Then, the overall score will be calculated by putting together these three weights for each parameter and the one with the best score will be the chosen parameter.

In [53]:
# Show parameters score
grouped_scores, best_params = train_validation_utilities.choose_best_params(hyp_res)
grouped_scores

,Split weight,RMSE weight,Frequency weight,Final score
Parameters,,,,
"(5, 0.2, gaussian, log)",0.5625,0.617771,1.00,0.347496
"(5, 0, gaussian, log)",0.5000,0.545394,0.25,0.068174


In [54]:
# Print best parameters
print(f"Best parameters: {best_params}")

Best parameters: (5, 0.2, 'gaussian', 'log')


### Cross validation

In [55]:
MODEL_TYPE = "cross_val"

In [56]:
# Get tuned parameters
params = train_validation_utilities.get_best_model_params(best_params, MODEL_NAME)
params

{'maxIter': [5], 'regParam': [0.2], 'family': ['gaussian'], 'link': ['log']}

In [57]:
# Perform cross validation
cv_train_result, cv_valid_result, cv_train_pred, cv_valid_pred = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

Split: [10/10]: 100%|██████████| 1/1 [00:01<00:00,  1.13s/it]


In [58]:
cv_train_result

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,cross_val,train,walk_forward_splits,base_and_most_corr_features_norm,1,"(20000, 5000)","[5, 0.2, gaussian, log]",870.331969,7.574777e+05,645.166678,0.013574,0.988279,0.988277,0.309795
1,GeneralizedLinearRegression,cross_val,train,walk_forward_splits,base_and_most_corr_features_norm,2,"(20000, 5000)","[5, 0.2, gaussian, log]",954.261262,9.106146e+05,738.286951,0.015710,0.987725,0.987723,0.274106
2,GeneralizedLinearRegression,cross_val,train,walk_forward_splits,base_and_most_corr_features_norm,3,"(20000, 5000)","[5, 0.2, gaussian, log]",1161.018848,1.347965e+06,887.093393,0.024684,0.977612,0.977608,0.291973
3,GeneralizedLinearRegression,cross_val,train,walk_forward_splits,base_and_most_corr_features_norm,4,"(20000, 5000)","[5, 0.2, gaussian, log]",1348.186777,1.817608e+06,1112.027689,0.035962,0.976219,0.976215,0.279018
4,GeneralizedLinearRegression,cross_val,train,walk_forward_splits,base_and_most_corr_features_norm,5,"(20000, 5000)","[5, 0.2, gaussian, log]",1201.358084,1.443261e+06,965.540057,0.033943,0.982212,0.982209,0.282047
5,GeneralizedLinearRegression,cross_val,train,walk_forward_splits,base_and_most_corr_features_norm,6,"(20000, 5000)","[5, 0.2, gaussian, log]",611.819177,3.743227e+05,427.308405,0.017772,0.984820,0.984817,0.292978
6,GeneralizedLinearRegression,cross_val,train,walk_forward_splits,base_and_most_corr_features_norm,7,"(20000, 5000)","[5, 0.2, gaussian, log]",179.110149,3.208045e+04,135.071211,0.006823,0.993602,0.993601,0.267860
7,GeneralizedLinearRegression,cross_val,train,walk_forward_splits,base_and_most_corr_features_norm,8,"(20000, 5000)","[5, 0.2, gaussian, log]",194.052843,3.765651e+04,138.658484,0.006799,0.993994,0.993993,0.281090
8,GeneralizedLinearRegression,cross_val,train,walk_forward_splits,base_and_most_corr_features_norm,9,"(20000, 5000)","[5, 0.2, gaussian, log]",501.420086,2.514221e+05,389.560896,0.017919,0.986480,0.986478,0.278210
9,GeneralizedLinearRegression,cross_val,train,walk_forward_splits,base_and_most_corr_features_norm,10,"(20000, 5000)","[5, 0.2, gaussian, log]",418.642806,1.752618e+05,349.947926,0.015910,0.991125,0.991124,0.286834


In [59]:
cv_valid_result

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,cross_val,valid,walk_forward_splits,base_and_most_corr_features_norm,1,"(20000, 5000)","[5, 0.2, gaussian, log]",1032.505654,1.066068e+06,823.533878,0.019276,0.847744,0.847622,0.309795
1,GeneralizedLinearRegression,cross_val,valid,walk_forward_splits,base_and_most_corr_features_norm,2,"(20000, 5000)","[5, 0.2, gaussian, log]",10960.929324,1.201420e+08,8041.059511,0.321360,-4.369544,-4.373844,0.274106
2,GeneralizedLinearRegression,cross_val,valid,walk_forward_splits,base_and_most_corr_features_norm,3,"(20000, 5000)","[5, 0.2, gaussian, log]",13917.335912,1.936922e+08,10669.519722,0.470179,-93.140048,-93.215435,0.291973
3,GeneralizedLinearRegression,cross_val,valid,walk_forward_splits,base_and_most_corr_features_norm,4,"(20000, 5000)","[5, 0.2, gaussian, log]",14241.104168,2.028090e+08,12366.473172,0.624681,-81.228460,-81.294309,0.279018
4,GeneralizedLinearRegression,cross_val,valid,walk_forward_splits,base_and_most_corr_features_norm,5,"(20000, 5000)","[5, 0.2, gaussian, log]",5853.310216,3.426124e+07,5513.065360,0.293564,-12.360351,-12.371050,0.282047
5,GeneralizedLinearRegression,cross_val,valid,walk_forward_splits,base_and_most_corr_features_norm,6,"(20000, 5000)","[5, 0.2, gaussian, log]",3851.819490,1.483651e+07,3546.354793,0.205909,-19.361102,-19.377407,0.292978
6,GeneralizedLinearRegression,cross_val,valid,walk_forward_splits,base_and_most_corr_features_norm,7,"(20000, 5000)","[5, 0.2, gaussian, log]",305.463130,9.330772e+04,229.193086,0.009992,0.895024,0.894940,0.267860
7,GeneralizedLinearRegression,cross_val,valid,walk_forward_splits,base_and_most_corr_features_norm,8,"(20000, 5000)","[5, 0.2, gaussian, log]",1998.797713,3.995192e+06,1851.814401,0.065483,0.330786,0.330250,0.281090
8,GeneralizedLinearRegression,cross_val,valid,walk_forward_splits,base_and_most_corr_features_norm,9,"(20000, 5000)","[5, 0.2, gaussian, log]",2271.432082,5.159404e+06,1898.854260,0.071028,-4.672269,-4.676812,0.278210
9,GeneralizedLinearRegression,cross_val,valid,walk_forward_splits,base_and_most_corr_features_norm,10,"(20000, 5000)","[5, 0.2, gaussian, log]",2073.920899,4.301148e+06,2043.932423,0.068153,-13.009683,-13.020901,0.286834


Let's see if the situation has improved by validating the model with the newly found parameters.

In [60]:
# Using single split method to validate the model
SPLITTING_METHOD = "single_split"

# Get splitting parameters based on the choosen splitting method
splitting_info = train_validation_utilities.get_splitting_params(SPLITTING_METHOD)
splitting_info

{'split_type': 'single_split', 'split_label': 'months', 'split_value': 1}

In [61]:
MODEL_TYPE = "tuned"

In [62]:
# Validate the model with the best parameters
tuned_train_results, tuned_valid_results, tuned_train_pred, tuned_valid_pred = train_validation_utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

In [63]:
tuned_train_results

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,tuned,train,single_split,base_and_most_corr_features_norm,"(67104, 2976)","[5, 0.2, gaussian, log]",3146.583221,9900985.966234,2551.228084,0.094549,0.942318,0.942315,0.440029


In [64]:
tuned_valid_results

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,tuned,valid,single_split,base_and_most_corr_features_norm,"(67104, 2976)","[5, 0.2, gaussian, log]",826.880193,683730.853633,670.745304,0.022783,-2.99521,-3.000589,0.440029


In [80]:
# Define the results to show
tuned_comparison_lst = [cv_valid_result, tuned_valid_results]

# Show the comparison table
tuned_comparison_lst_df = pd.concat([train_validation_utilities.model_comparison(results, model_info, evaluator_lst) for results in tuned_comparison_lst])
tuned_comparison_lst_df

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,cross_val,valid,walk_forward_splits,base_and_most_corr_features_norm,"[5, 0.2, gaussian, log]",5650.661859,5.803561e+07,4698.380061,0.214963,-22.60679,-22.625695,0.284391
0,GeneralizedLinearRegression,tuned,valid,single_split,base_and_most_corr_features_norm,"[5, 0.2, gaussian, log]",826.880193,6.837309e+05,670.745304,0.022783,-2.99521,-3.000589,0.440029


## Trained
At this point it is possible to train the final model over the entire train / validation set in order to use it in the final step, that is, to make predictions about the data contained in the test set.

In [88]:
MODEL_TYPE = "final"

In [89]:
# Train the final model on the whole train / validation set
final_train_results, final_model, final_train_pred = train_validation_utilities.evaluate_trained_model(df, params, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

In [90]:
final_train_results

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,final,train,whole_train_valid,base_and_most_corr_features_norm,"[5, 0.2, gaussian, log]",3083.328048,9.506912e+06,2471.249673,0.091483,0.942279,0.942276,0.330757


# Comparison table
Visualization of model performance at various stages of train / validation

In [91]:
# Concatenate final results into Pandas dataset
final_comparison_lst_df = pd.DataFrame(pd.concat([default_comparison_lst_df, tuned_comparison_lst_df, final_train_results], ignore_index=True))
final_comparison_lst_df

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default,valid,single_split,base_features,"[25, 0]",2.683494,7.201143e+00,1.993841,0.000067,0.999958,0.999958,1.813228
1,GeneralizedLinearRegression,default,valid,single_split,base_and_most_corr_features,"[25, 0]",2.746864,7.545262e+00,2.049422,0.000069,0.999956,0.999956,1.047293
2,GeneralizedLinearRegression,default,valid,single_split,base_and_least_corr_features,"[25, 0]",5.350591,2.862883e+01,4.869163,0.000165,0.999833,0.999832,1.057417
3,GeneralizedLinearRegression,default_norm,valid,single_split,base_features_norm,"[25, 0]",3225.273767,1.040239e+07,3159.373044,0.106917,-59.783760,-59.865596,0.752858
4,GeneralizedLinearRegression,default_norm,valid,single_split,base_and_most_corr_features_norm,"[25, 0]",829.037237,6.873027e+05,672.362273,0.022838,-3.016081,-3.021488,0.551809
5,GeneralizedLinearRegression,default_norm,valid,single_split,base_and_least_corr_features_norm,"[25, 0]",1586.559230,2.517170e+06,1559.544555,0.052708,-13.708452,-13.728255,0.609058
6,GeneralizedLinearRegression,cross_val,valid,walk_forward_splits,base_and_most_corr_features_norm,"[5, 0.2, gaussian, log]",5650.661859,5.803561e+07,4698.380061,0.214963,-22.606790,-22.625695,0.284391
7,GeneralizedLinearRegression,tuned,valid,single_split,base_and_most_corr_features_norm,"[5, 0.2, gaussian, log]",826.880193,6.837309e+05,670.745304,0.022783,-2.995210,-3.000589,0.440029
8,GeneralizedLinearRegression,final,train,whole_train_valid,base_and_most_corr_features_norm,"[5, 0.2, gaussian, log]",3083.328048,9.506912e+06,2471.249673,0.091483,0.942279,0.942276,0.330757


# Model accuracy

Since predicting the price accurately is very difficult I also saw how good the models are at predicting whether the price will go up or down in this way:

For each prediction let's consider the actual market-price, next-market-price and our predicted next-market-price (prediction).
I compute whether the current prediction is correct (1) or not (0):

$$ 
prediction\_is\_correct
= 
\begin{cases}
0 \text{ if [(market-price > next-market-price) and (market-price < prediction)] or [(market-price < next-market-price) and (market-price > prediction)]} \\
1 \text{ if [(market-price > next-market-price) and (market-price > prediction)] or [(market-price < next-market-price) and (market-price < prediction)]}
\end{cases}
$$

After that I count the number of correct prediction:
$$ 
correct\_predictions
= 
\sum_{i=0}^{total\_rows} prediction\_is\_correct
$$

Finally I compute the percentage of accuracy of the model:
$$
\\ 
accuracy 
= 
(correct\_predictions / total\_rows) 
* 100
$$

In [92]:
# Convert the pandas dataset to a PySpark dataset
best_default_pred_spark = spark.createDataFrame(best_default_predictions)
validated_pred_spark = spark.createDataFrame(tuned_valid_pred)

# Compute model accuracy
default_accuracy = train_validation_utilities.model_accuracy(best_default_pred_spark)
validated_accuracy = train_validation_utilities.model_accuracy(validated_pred_spark)

# Shows whether features are normalised or not
if FEATURES_NORMALIZATION:
    NEW_CHOSEN_FEATURES_LABEL = CHOSEN_FEATURES_LABEL + "_norm"
    CHOSEN_FEATURES_LABEL = NEW_CHOSEN_FEATURES_LABEL

# Saving accuracy data into dataframe
accuracy_data = {
    'Model': MODEL_NAME,
    'Features': CHOSEN_FEATURES_LABEL,
    'Splitting': SPLITTING_METHOD,
    'Accuracy (default)': default_accuracy,
    'Accuracy (tuned)': validated_accuracy
}
accuracy_data_df = pd.DataFrame(accuracy_data, index=['Model'])

print(f"Percentage of correct predictions for {MODEL_NAME} with {CHOSEN_FEATURES_LABEL} and {SPLITTING_METHOD} (default): {default_accuracy:.2f}%")
print(f"Percentage of correct predictions for {MODEL_NAME} with {CHOSEN_FEATURES_LABEL} and {SPLITTING_METHOD} (tuned): {validated_accuracy:.2f}%")

Percentage of correct predictions for GeneralizedLinearRegression with base_and_most_corr_features_norm and single_split (default): 46.71%
Percentage of correct predictions for GeneralizedLinearRegression with base_and_most_corr_features_norm and single_split (tuned): 46.67%


In [93]:
# Concatenate default and tuned results
default_tuned_results = [best_default_results, tuned_valid_results]
default_tuned_results_df = pd.concat([train_validation_utilities.model_comparison(results, model_info, evaluator_lst) for results in default_tuned_results])
default_tuned_results_df

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default_norm,valid,single_split,base_and_most_corr_features_norm,"[25, 0]",829.037237,687302.740443,672.362273,0.022838,-3.016081,-3.021488,0.551809
0,GeneralizedLinearRegression,tuned,valid,single_split,base_and_most_corr_features_norm,"[5, 0.2, gaussian, log]",826.880193,683730.853633,670.745304,0.022783,-2.995210,-3.000589,0.440029


# Saving final results

In [94]:
# Save all final comparison results
final_comparison_lst_df.to_csv(ALL_MODEL_RESULTS, index=False)

In [95]:
# Save relevant results (default and tuned results)
default_tuned_results_df.to_csv(REL_MODEL_RESULTS, index=False)

In [96]:
# Saving accuracy results
accuracy_data_df.to_csv(MODEL_ACCURACY_RESULTS, index=False)

In [97]:
# Saving final model
final_model.write().overwrite().save(MAIN_DIR + '/models/' + MODEL_NAME)

In [98]:
# Export notebook in html format (remember to save the notebook and change the model name)
if LOCAL_RUNNING:
    !jupyter nbconvert --to html 5-single-split_{MODEL_NAME}.ipynb --output 5-single-split_{MODEL_NAME} --output-dir='./exports'

d:\Documents\Repository\BDC\project\.bdc\Lib\site-packages\traitlets\traitlets.py:2548: FutureWarning: Supporting extra quotes around strings is deprecated in traitlets 5.0. You can use './exports' instead of "'./exports'" if you require traitlets >=5.
  warn(
[NbConvertApp] Converting notebook 5-single-split_GeneralizedLinearRegression.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 1 image(s).
[NbConvertApp] Writing 461602 bytes to exports\5-single-split_GeneralizedLinearRegression.html
